In [1]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.tasks.python.vision import FaceLandmarkerOptions, FaceLandmarker
import numpy as np

# Image

In [11]:
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='face_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE,
    num_faces=2,
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=True,
)

In [46]:
with FaceLandmarker.create_from_options(options) as face_landmarker:
    image = mp.Image.create_from_file('D:/ATHARV/W/CDAC/CODE/PRACTICE/Test_Images/face_cry.jpg')
    result = face_landmarker.detect(image)

    annotated_image = image.numpy_view().copy()

    if result.face_landmarks:
        for landmark in result.face_landmarks[0]:
            x = int(landmark.x * annotated_image.shape[1])
            y = int(landmark.y * annotated_image.shape[0])
            cv2.circle(annotated_image, (x, y), 2, (0, 255, 0), -1)

    if result.face_blendshapes:
        top_shapes = sorted(
            result.face_blendshapes[0],
            key=lambda c: c.score,
            reverse=True
        )[0:10]

        for i, shape in enumerate(top_shapes):
            text = f"{shape.category_name}: {shape.score:.2f}"
            cv2.putText(annotated_image, text, (10, 30 + i * 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

In [47]:
display_image = cv2.resize(annotated_image, (1024, 768))
cv2.imshow('Static Face Mesh with Blendshapes', display_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Videos

In [2]:
model_path = "face_landmarker.task"
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=True,
    num_faces=1,
    running_mode=VisionRunningMode.VIDEO
)

face_landmarker = FaceLandmarker.create_from_options(options)

In [4]:
cam = cv2.VideoCapture(0)
while cam.isOpened():
    success, frame = cam.read()
    if not success:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    timestamp = int(cv2.getTickCount() / cv2.getTickFrequency() * 1000)

    result = face_landmarker.detect_for_video(mp_image, timestamp)

    if result.face_landmarks:
        for face_landmarks in result.face_landmarks:
            for landmark in face_landmarks:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

    if result.face_blendshapes:
        top_shapes = sorted(
            result.face_blendshapes[0],
            key=lambda c: c.score,
            reverse=True
        )[:10]
        for i, shape in enumerate(top_shapes):
            text = f"{shape.category_name}: {shape.score:.2f}"
            cv2.putText(frame, text, (10, 30 + i * 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)

    cv2.imshow("Face Landmarks & Blendshapes - VIDEO Mode", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # ESC key to exit
        break

cam.release()
cv2.destroyAllWindows()
